In [ ]:
#load the example files I have

import os 
import json
samples = list()

folder = "my_examples"

for _file in os.listdir(folder):
    filepath = os.path.join(folder, _file)
    with open(filepath, 'rb') as f:
        data = json.load(f)
        _input ={
            "primarySourceText": data["primarySourceText"],
            "citation": data["citation"]
        }
        samples.append({
            "input": _input, 
            "output": data
        })
samples[0]

In [ ]:
import openai 
import asyncio
openai.api_key = input("what is gpt api key")
model_engine = "gpt-3.5-turbo"

general_prompt = f"""You are a game designer assistance for me who is trying to create an interactive Odyssey game. 
Given primary text, you are to evolve it into game components that I can process and feed to the user.
Each primary text is linked to a Node. A node is made up of a primarySourceText, a citation (most provided) as well as an entranceDescription (what people see before encountering anyone, and component (a list of components).

Each component represents a seperate part of the island and has the following fields: status (friendly/nuetral/hostile), fightingStrength: 0-5, intelligence: 0-5, agression: 0-5, explorerDescription: what you see right before encountering this creature, exhibitsXenia: if this creature will feed and house the character; who: what this creature is; fightingDetails: any associated fighting details; sampleBenefits: a mapper for each quality of food/fame/gold/shipQaulity to an amount of how it increases and an example of how it happens.

Here are some examples - use the baseline numbers for stuff like fame here to inform your future choices:
""" + "\n\n\n__________________________\n\n\n".join(["input:\n"+str(s["input"]) + "\noutput:\n" + str(s["output"]) for s in samples]) + "\n\n\nRemember, the response should be JSON serializable. Use double quotes everywhere."
general_prompt

In [ ]:
#get input files for gpt
_inputs = list()
folder = "primary_sources_for_gpt"
input_list = os.listdir(folder)
for _file in input_list:
    filepath = os.path.join(folder, _file)
    with open(filepath, 'rb') as f:
        d = json.load(f)
        _inputs.append(d)


#get completions for each of these
completions = list()
for specific_input in _inputs:
    print("input:\n"+str(specific_input))
    completion = openai.ChatCompletion.acreate(
                model = model_engine,
                messages = [{"role": "system", "content": general_prompt}] +[{"role": "user", "content": "input:\n"+str(specific_input)}]
                            
            )
    completions.append(completion)

In [ ]:
completions = [await c for c in completions]
completions

In [ ]:
# json.loads(completions[0].choices[0].message.content)
import simplejson
output_folder = "gpt_examples" 

ignore_names = ["charybdis_scylla.json", "lamus.json", "sirens.json"]
for idx, filename in enumerate(input_list):
    try:
        this_content = json.loads(completions[idx].choices[0].message.content)
        print("good", completions[idx].choices[0].message.content)
    except:
        print("bad", completions[idx].choices[0].message.content)
        continue
        
    with open(os.path.join(output_folder, filename), 'w') as f:
        simplejson.dump(this_content, f, encoding='utf-8', ignore_nan=True)

    

In [ ]:
# !pip install simplejson